In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [ ]:

!wget "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"
!unzip "/content/kagglecatsanddogs_3367a.zip"

In [ ]:
#@title Setting up Dataset
!wget "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"
!unzip "/content/kagglecatsanddogs_3367a.zip"
Data_dir="/content/PetImages"
categories= ["Dog","Cat"]
image_size=50
def create_training_data(Data_path,size_of_image):
  training_data=[]
  for category in categories:
    path=os.path.join(Data_path,category) #get path of each class
    class_num=categories.index(category)
    for img in os.listdir(path):
      try:
        image_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        image_array=cv2.resize(image_array,(size_of_image,size_of_image))
        training_data.append([image_array,class_num])
      except Exception as e:
        pass
  return training_data
C_and_D=create_training_data(Data_dir,image_size)
random.shuffle(C_and_D)
for sample in C_and_D[:10]:
  print(sample[1])

X=[]
y=[]
for features, labels in C_and_D:
    X.append(features)
    y.append(labels)

X2=np.array(X)
X=np.array(X).reshape(-1,image_size,image_size,1)
pickle_out=open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out=open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()
!cp "/content/X.pickle" "/content/drive/MyDrive/DataSets/Cats and dogs/"
!cp "/content/y.pickle" "/content/drive/MyDrive/DataSets/Cats and dogs/"

In [ ]:
!cp "/content/drive/MyDrive/DataSets/Cats and dogs/X.pickle" "/content/"
!cp "/content/drive/MyDrive/DataSets/Cats and dogs/y.pickle" "/content/"

In [ ]:
pickle_in=open("X.pickle","rb")
X=pickle.load(pickle_in)
pickle_in=open("y.pickle","rb")
y=pickle.load(pickle_in)
X=X/255.0
y=np.array(y)
print(len(y),len(X))

24946 24946


In [ ]:
# dense_layers=[0, 1, 2] #num of dense layers used
# layer_sizes=[32, 64, 128] #also knows as nodes
# conv_layers=[1, 2, 3] #num of conv layers used

dense_layers=[0] #num of dense layers used
layer_sizes=[64] #also knows as nodes
conv_layers=[3] #num of conv layers used

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME="{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboad=TensorBoard(log_dir='/content/logs/{}'.format(NAME))

            model =Sequential()
            model.add(Conv2D(layer_size,(3,3),input_shape=X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))
                model.add(Dropout(0.2))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss="binary_crossentropy",
                        optimizer="adam",
                        metrics=['accuracy'])

            model.fit(X,y,batch_size=32, epochs=10, validation_split=0.3,callbacks=[tensorboad])


Epoch 1/10
546/546 [==============================] - 100s 182ms/step - loss: 0.6585 - accuracy: 0.6005 - val_loss: 0.6068 - val_accuracy: 0.6730
Epoch 2/10
546/546 [==============================] - 99s 181ms/step - loss: 0.5882 - accuracy: 0.6909 - val_loss: 0.5562 - val_accuracy: 0.7171
Epoch 3/10
546/546 [==============================] - 101s 186ms/step - loss: 0.5406 - accuracy: 0.7249 - val_loss: 0.5306 - val_accuracy: 0.7377
Epoch 4/10
546/546 [==============================] - 100s 183ms/step - loss: 0.5070 - accuracy: 0.7529 - val_loss: 0.5065 - val_accuracy: 0.7575
Epoch 5/10
546/546 [==============================] - 100s 183ms/step - loss: 0.4851 - accuracy: 0.7669 - val_loss: 0.4781 - val_accuracy: 0.7746
Epoch 6/10
546/546 [==============================] - 101s 185ms/step - loss: 0.4603 - accuracy: 0.7838 - val_loss: 0.4829 - val_accuracy: 0.7676
Epoch 7/10
546/546 [==============================] - 99s 182ms/step - loss: 0.4370 - accuracy: 0.7958 - val_loss: 0.4654 - v

In [ ]:
model.save("64x3-CNN.model")

INFO:tensorflow:Assets written to: 64x3-CNN.model/assets


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
from tensorboard import notebook
notebook.list()

In [ ]:
!cp "/content/drive/MyDrive/DataSets/Cats and dogs/cat1.jpeg" "/content/"
!cp "/content/drive/MyDrive/DataSets/Cats and dogs/dog1.jpeg" "/content/"
!cp "/content/drive/MyDrive/DataSets/Cats and dogs/dog2.jpeg" "/content/"
!cp "/content/drive/MyDrive/DataSets/Cats and dogs/dog3.jpeg" "/content/"

In [ ]:
def prepare(filepath):
    IMG_SIZE=50
    img_array=cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
    new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,1)

model=tf.keras.models.load_model("64x3-CNN.model")

In [ ]:
print(model.predict(prepare("/content/dog1.jpeg")))
print(model.predict(prepare("/content/cat1.jpeg")))